## Proces wyodrębnienia przyzwoitych pseudo-masek

#### Importowanie bibliotek

In [1]:
from pathlib import Path
import shutil
import csv

#### Funkcja `load_ids`
Wczytanie listy identyfikatorów do przeprowadzenia kopiowania

In [2]:
def load_ids(csv_path, id_field = "image_id"):
    ids = set()
    with csv_path.open(newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            ids.add(Path(row[id_field]).stem)
    return ids

#### Funkcja `build_file_index`
Funkcja tworzy słownik indeksujący wszystkie pliki w katalogu w którym kluczem jest nazwa pliku bez rozszerzenia, a wartością pełna ścieżka.

In [3]:
def build_file_index(src_dir):
    index = {}
    for p in src_dir.iterdir():
        if p.is_file():
            index[p.stem] = p
    return index

#### Funkcja `copy_selected_files`
Funkcja odpowiadająca za przeprowadzenie kopiowania wybranych plików

In [4]:
def copy_selected_files(src_dir, dst_dir, selected_ids, file_index,):
    dst_dir.mkdir(parents=True, exist_ok=True)
    for stem in selected_ids:
        src = file_index.get(stem)
        
        if src and src.is_file():
            shutil.copy2(src, dst_dir / src.name)
        else:
            print(F"Brak pliku: {stem}")
    print(f"Skopiowano {len(selected_ids)} plików z `{src_dir.name}` do `{dst_dir}`")

#### Funkcja `process_copy_classification`
Funkcja grupująca wywołanie wszystkich funkcji potrzebnych do poprawnego przeprowadzenia kopiowania plików

In [ ]:
def process_copy_classification(images_dir, csv_path, output_base, split_name = "", id_field = "image_id"):
    ids = load_ids(csv_path, id_field)

    idx_imgs  = build_file_index(images_dir)

    if split_name:
        out_imgs  = output_base / split_name
    else:
        out_imgs  = output_base

    copy_selected_files(images_dir, out_imgs, ids, idx_imgs)